In [41]:
#In this case, we are going to try to predict whether a sentence comes from Alice in Wonderland by Lewis Carroll 
#or Persuasion by Jane Austen.
#We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. 
#In this case, we'll try Random Forests, SVM, and KNN.

In [42]:
#Our feature-generation approach will be something called BoW, or Bag of Words. 
#BoW is quite simple: For each sentence, we count how many times each word appears. 
#We will then use those counts as features.

In [43]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [44]:
#Steps to use: 
#1. Load the clean data into variables
#2. Identify the chapter that is being cleaned
#3. Clean the text. Get rid of double dash and punctuation like question marks
#4. Tokenized the data
#5. Label the data
#6. Set up the bags (Bag of words) for each book
#7. Combine bags to create a set of unique words
#8. Create our data frame with features (which are the unique words created in step 7)
#9. Run the bag of words features (created in step 8) through variours models i.e. random forest, 
    #Logistic regression and SVN.

In [45]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [46]:
# Parse the cleaned novels. This can take a bit. 
#aka tokenized the data
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [47]:
# Group into sentences.
#aka label the data
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [48]:
#Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording
#how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our 
#feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll 
#only use the 2000 most common words for each text.

In [50]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(500)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [51]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,pardon,ill,company,enjoyment,wonder,tart,subject,sensible,card,what,...,but,late,regret,for,morrow,value,fail,smile,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [52]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

/Users/karenmcgee/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9796238244514106

Test set score: 0.8890977443609023


In [53]:
#Overfitting is a known problem when using bag of words, since it basically involves throwing a massive 
#number of features at a model – some of those features (in this case, word frequencies) will capture noise 
#in the training set. Since overfitting is also a known problem with Random Forests, the divergence between 
#training score and test score is expected.

In [54]:
#Let's try a technique with some protection against overfitting due to extraneous features – logistic regression 
#with ridge regularization (from ridge regression, also called L2 regularization).

In [55]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 774) (3190,)
Training set score: 0.9423197492163009

Test set score: 0.9163533834586466


/Users/karenmcgee/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [56]:
#Logistic regression performs a bit better than the random forest.

In [57]:
#And finally, let's see what gradient boosting can do:

In [58]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8874608150470219

Test set score: 0.8745300751879699


In [59]:
#Looks like logistic regression is the winner, but there's room for improvement.

In [60]:
#What if we feed the model a different novel by Jane Austen, like Emma? Will it be able to distinguish Austen 
#from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?
#First, we need to process Emma the same way we processed the other data, and combine it with the Alice data:

In [61]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [63]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [64]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [65]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [66]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.691648023862789


col_0,Austen,Carroll
row_0,,
Austen,1561,108
Carroll,719,294


In [ ]:
#Well look at that! NLP approaches are generally effective on the same type of material as they were trained on. 
#It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.

In [ ]:
#Challenge 0:
#Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to 
#improve performance.

In [67]:
from sklearn.svm import LinearSVC

svm = LinearSVC(C=2) #update the parameter of c to improve performance
train = svm.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

#compare the training and testing results using cross validation
from sklearn.model_selection import cross_val_score
cv_results_train = cross_val_score(svm, X_train, y_train, cv=5)
cv_results_test = cross_val_score(svm, X_test, y_test, cv=5)
print('Training set score:', cv_results_test)


(3190, 774) (3190,)
Training set score: 0.9639498432601881

Test set score: 0.8984962406015038
Training set score: [0.89461358 0.87793427 0.88235294 0.87294118 0.89411765]


In [68]:
#Challenge 1:
#Find out whether your new model is good at identifying Alice in Wonderland vs any other work, 
#Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from 
#the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [69]:
#find a title for the newbook
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [70]:
# Clean the Emma data.
#replace Emma with a new book's name
moby_dick = gutenberg.raw('melville-moby_dick.txt')
moby_dick = re.sub(r'VOLUME \w+', '', moby_dick)
moby_dick = re.sub(r'CHAPTER \w+', '', moby_dick)
moby_dick = text_cleaner(moby_dick)
print(moby_dick[:100])

ETYMOLOGY. (Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher threadbare in c


In [71]:
# Parse our cleaned data.
# moby dick is quite long, let's cut it down to the same length as Austen.
moby_dick_len = moby_dick[0:len(persuasion)]
moby_dick_doc = nlp(moby_dick_len)

In [72]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
moby_dick_sents = [[sent, "Moby_Dick"] for sent in moby_dick_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
moby_dick_sents = moby_dick_sents[0:len(persuasion_sents)]

In [75]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
moby_dick_sentences = pd.DataFrame(moby_dick_sents)
moby_dick_bow = bow_features(moby_dick_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
done


In [78]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_moby_dick_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    moby_dick_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_moby_dick_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Moby_Dick'] * moby_dick_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_moby_dick_test, y_moby_dick_test))
lr_moby_dick_predicted = lr.predict(X_moby_dick_test)
pd.crosstab(y_moby_dick_test, lr_moby_dick_predicted)


Test set score: 0.06306306306306306


col_0,Austen,Carroll
row_0,,
Carroll,719,294
Moby_Dick,2881,768


# Overall my model was able to successfully identify other works such as my example of moby dick